In [1]:
from __future__ import print_function
import binascii
import struct
from PIL import Image
import numpy as np
import scipy
import scipy.misc
import scipy.cluster
import cv2
import numpy as np
import pickle
from collections import Counter
import operator


In [2]:
def most_common_color(image):
    ar = np.asarray(face)
    shape = ar.shape
    ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)

    codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)

    vecs, dist = scipy.cluster.vq.vq(ar, codes)         
    counts, bins = scipy.histogram(vecs, len(codes))   
    index_max = scipy.argmax(counts)                    
    peak = codes[index_max]
    return peak

In [3]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.2):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img,0,0,500,500
    
    for (x,y,w,h) in faces:
        x = x - 20
        w = w + 20
        y = y - 20
        h = h + 20
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
    roi_color = cv2.flip(roi_color,1)
    x_izquierda=x
    x_derecha=x+w
    y_arriba=y
    y_abajo=y+h
    return roi_color,x_izquierda,x_derecha,y_abajo,y_arriba


In [4]:
def suma(lista):
    high = []
    for x in lista:
        high.append(x+30)
    return high

In [5]:
def resta(lista):
    low = []
    for x in lista:
        low.append(x-20)
    return low

In [6]:
cap = cv2.VideoCapture(0)
color = []
NUM_CLUSTERS = 6
i=0
while True:
    i+=1
    ret, frame = cap.read()
    
    
    #catch the zone where is the face
    face,x_izquierda,x_derecha,y_abajo,y_arriba = face_detector(frame)
    #take the color of the skin (most used)
    most_common_color_rgb = most_common_color(face).tolist()
    high = tuple(suma(most_common_color_rgb))
    low = tuple(resta(most_common_color_rgb))
    #apply a mask on that color based on a range of 20
    mask = cv2.inRange(frame, low, high)
    #mask is the new "Final photo"
    cv2.rectangle(mask,(0,0),(1500,y_abajo+20),(255,255,0),-1)
    thresh = cv2.erode(mask, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    ######## find contours
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if len(contours) != 0:
        # draw in blue the contours that were founded
        cv2.drawContours(mask, contours, -1, 255, 3)

        #find the biggest area
        c = max(contours, key = cv2.contourArea)

        x,y,w,h = cv2.boundingRect(c)
        # draw the book contour (in green)
        
        if h>50 and h<130 and w > 70 and w<130:
            cv2.rectangle(mask,(x,y),(x+w,y+h),(255,255,0),9)
            add_h=224-h
            add_w=224-w
            hand = frame[x-(add_w//2):y+h+add_h, x+(add_h//2):x+w+add_w]
            cv2.imwrite("hand"+str(i)+".png", hand)
    
    cv2.imshow("patapa",mask)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()